# Class Schedule 

* Joel Philippe Melo Figueiras, nº 20809
* Nuno Miguel Carvalho Araújo, nº 20078
* Vítor Hugo Sá Machado, nº 21158

#### Por favor, importar todas as dependências antes de executar qualquer algoritmo.

In [ ]:
## Imports
from array import *
from csp import *

# %matplotlib inline
# Hide warnings in the matplotlib sections
import math
import warnings
warnings.filterwarnings("ignore")

#### Por favor, importar todas as funções auxiliares antes de executar qualquer algoritmo.

In [ ]:
## Funções auxiliares

# Atribuir o máximo de três UCs diárias (1-4 | 5-8 | 9-12 | 13-16 | 17-20)
def max_three(*values):
    dictVal = {}
    for value in values:
        section = math.ceil(value / 4)
        content = dictVal.get(section, 0)
        dictVal[section] = content + 1
        if (dictVal[section] > 3): return False
    return True

# Atribuir o mínimo de uma UC semanal online
def min_one_online(*values):
    for value in values:
        if(value == 50):
            return True
    return False

# Atribuir o máximo de duas UCs semanais online
def max_two_online(*values):
    counter = 0
    for value in values:
        if(value == 50): counter = counter + 1
    if (counter > 2): return False
    return True

# Atribuir um espaço vazio entre UC online e presencial
def space_between_ucs(*values):
    split = []
    lenValues = len(values)
    n = int(lenValues / 5)
    posArray = 0
    posArrayNext = posArray + 1

    for i in range(n):
        start = int(i*lenValues/n)
        end = int((i+1)*lenValues/n)
        split.append(values[start:end])

    for array in split:
        if(posArray % 2 == 0):
            posCurrent = 0
            posCurrent2 = 0
            for pos in array:
                if(split[posArrayNext][posCurrent] == 50):
                    posCurrent2 = 0
                    for pos2 in array:
                        if((pos2+1 == pos) or (pos2-1 == pos)):
                            if(split[posArrayNext][posCurrent2] != 50):
                                return False
                        posCurrent2 = posCurrent2 + 1
                posCurrent = posCurrent + 1
        posArray = posArray + 1
        posArrayNext = posArrayNext + 1
    return True
        
# Conciliar o professor ao respetivo bloco em que a UC está inserida 
def addTeachers(T, result):
    auxDict = {}
    if(result):
        for t in T:
            for res in result:
                if('B.' in res):
                    if(t[t.index("."):] == res[res.index("."):]):
                        auxDict[t] = result[res]
        return(auxDict)
    else: print("ERRO NA ATRIBUIÇÃO DOS PROFESSORES")

# Dicionário com o resultado final das atribuições
def finalOutput(solver, teachers):
    result = {}
    if(solver):
        for s in solver:
            if('B.' in s):
                if(s not in result):
                    result[s] = ['B'+str(solver[s])]
        for s in solver:
            if('C.' in s):
                for r in result:
                    if((s[s.index("."):] == r[r.index("."):])):
                        result[r].append('CR'+str(solver[s]))
        for t in teachers:
            for r in result:
                if(t[t.index("."):] == r[r.index("."):]):
                    result[r].append(t[:t.index(".")])
        
        for res in sorted(result.keys()):
            print("\n"+res, end=": ")
            for var in result[res]:
                print(var, end=" ")
    else: print("ERROR")

#### Atribuição de um bloco por UC para uma turma

No primeiro teste iremos executar o algoritmo atribuindo a uma turma apenas um bloco por UC.  
Existem três professores, cada um responsável por diferentes UCs.  
Existem também três salas por onde poderão ser distribuídas as UCs, sendo que cada sala tem UCs especificas.

Tempo de conclusão do algoritmo: 0,7 segundos.

In [ ]:
## Atribuição de um bloco por UC para uma turma

schedules = set(range(1, 21))
domains = {}

## ClassRooms
# 0 -> PI, MD | 1 -> C, RC | 2 -> LI | 50 -> Aulas online
CR = [0, 1, 2, 50]

## Teachers
# T0 -> PI1, LI1 | T1 -> MD1, C1 | T2 -> RC1
TESI1 = ['T0.PI1', 'T0.LI1', 'T1.MD1', 'T1.C1', 'T2.RC1']

## VARIABLES
# Classes
ESI1B = ['ESI1B.PI1', 'ESI1B.MD1', 'ESI1B.C1', 'ESI1B.RC1', 'ESI1B.LI1']    # Armazenamento do bloco em que a UC é dada
ESI1C = ['ESI1C.PI1', 'ESI1C.MD1', 'ESI1C.C1', 'ESI1C.RC1', 'ESI1C.LI1']    # Armazenamento da sala em que a UC é dada

## Atribuição de domínios às variáveis
variables = set (ESI1B)
for var in variables:
    domains[var] = schedules

# ClassRooms
domains[ESI1C[0]] = set((CR[0], CR[3])) # A UC 'PI1' tem de ser dada na sala 0 ou online
domains[ESI1C[1]] = set((CR[0], CR[3])) # A UC 'MD1' tem de ser dada na sala 0 ou online
domains[ESI1C[2]] = set((CR[1], CR[3])) # A UC 'C1' tem de ser dada na sala 1 ou online
domains[ESI1C[3]] = set((CR[1], CR[3])) # A UC 'RC1' tem de ser dada na sala 1 ou online
domains[ESI1C[4]] = set((CR[2], CR[3])) # A UC 'LI1' tem de ser dada na sala 2 ou online

# Constraints
constraints = [
    #Rules
    Constraint((ESI1B), max_three),                 # Máximo 3 UCs por dia
    Constraint((ESI1C), min_one_online),            # Mínimo 1 UC online
    Constraint((ESI1C), max_two_online),            # Máximo 2 UCs online
    Constraint((ESI1B + ESI1C), space_between_ucs),  # Atribuir um bloco vazio entre uma UC online e presencial

    # Classes
    Constraint((ESI1B), all_diff_constraint), # Todas as UCs tê de ser em blocos diferentes
]

# Class scheduling
class_scheduling = NaryCSP(domains, constraints)

# print variables
print(class_scheduling.variables)

# Result
solver = ac_solver(class_scheduling, arc_heuristic=sat_up)
teachers = addTeachers(TESI1, solver)

# Output
finalOutput(solver, teachers)

#### Atribuição de um bloco por UC para três turmas

No segundo teste iremos executar o algoritmo atribuindo a três turma apenas um bloco por UC.  
Existem sete professores, cada um responsável por diferentes UCs.  
Existem também cinco salas por onde poderão ser distribuídas as UCs, sendo que cada sala tem UCs especificas.

Tempo de conclusão do algoritmo: 2,3 segundos.

In [ ]:
## Atribuição de um bloco por UC para três turmas

schedules = set(range(1, 21))
domains = {}

## ClassRooms
# 0 -> PI, MD, POO | 1 -> C, RC, AMS | 2 -> LI, FF, PES | 3 -> AAD, IA, PDM, ISI | 4 -> SETR, PA
CR = [0, 1, 2, 3, 4, 50]

## Teachers
# T0 -> PI, LI | T1 -> MD, C | T2 -> RC, POO, AMS | T3 -> FF, PES | T4 -> AAD, IA | T5 -> PDM, ISI | T6 -> SETR, PA
TESI1 = ['T0.PI1', 'T0.LI1', 'T1.MD1', 'T1.C1', 'T2.RC1']       
TESI2 = ['T2.POO1', 'T2.AMS1', 'T3.FF1', 'T3.PES1' ,'T4.AAD1']  
TESI3 = ['T4.IA1', 'T5.PDM1', 'T5.ISI1', 'T6.SETR1' ,'T6.PA1']  


## VARIABLES
# Classes
    # 1º ano ESI
ESI1B = ['ESI1B.PI1', 'ESI1B.MD1', 'ESI1B.C1', 'ESI1B.RC1', 'ESI1B.LI1']    # Armazenamento do bloco em que a UC é dada
ESI1C = ['ESI1C.PI1', 'ESI1C.MD1', 'ESI1C.C1', 'ESI1C.RC1', 'ESI1C.LI1']    # Armazenamento da sala em que a UC é dada
    # 2º ano ESI
ESI2B = ['ESI2B.POO1', 'ESI2B.AMS1', 'ESI2B.FF1', 'ESI2B.PES1', 'ESI2B.AAD1']    # Armazenamento do bloco em que a UC é dada
ESI2C = ['ESI2C.POO1', 'ESI2C.AMS1', 'ESI2C.FF1', 'ESI2C.PES1', 'ESI2C.AAD1']    # Armazenamento da sala em que a UC é dada
    # 3º ano ESI
ESI3B = ['ESI3B.IA1', 'ESI3B.PDM1', 'ESI3B.ISI1', 'ESI3B.SETR1', 'ESI3B.PA1']    # Armazenamento do bloco em que a UC é dada
ESI3C = ['ESI3C.IA1', 'ESI3C.PDM1', 'ESI3C.ISI1', 'ESI3C.SETR1', 'ESI3C.PA1']    # Armazenamento da sala em que a UC é dada

## Atribuição de domínios às variáveis
variables = set (ESI1B + ESI2B + ESI3B)
for var in variables:
    domains[var] = schedules

# ClassRooms
domains[ESI1C[0]] = set((CR[0], CR[5])) # A UC 'PI1' tem de ser dada na sala 0
domains[ESI1C[1]] = set((CR[0], CR[5])) # A UC 'MD1' tem de ser dada na sala 0
domains[ESI2C[0]] = set((CR[0], CR[5])) # A UC 'POO1' tem de ser dada na sala 0

domains[ESI1C[2]] = set((CR[1], CR[5])) # A UC 'C1' tem de ser dada na sala 1
domains[ESI1C[3]] = set((CR[1], CR[5])) # A UC 'RC1' tem de ser dada na sala 1
domains[ESI2C[1]] = set((CR[1], CR[5])) # A UC 'AMS' tem de ser dada na sala 1

domains[ESI1C[4]] = set((CR[2], CR[5])) # A UC 'LI1' tem de ser dada na sala 2
domains[ESI2C[2]] = set((CR[2], CR[5])) # A UC 'FF' tem de ser dada na sala 2
domains[ESI2C[3]] = set((CR[2], CR[5])) # A UC 'PES' tem de ser dada na sala 2

domains[ESI2C[4]] = set((CR[3], CR[5])) # A UC 'AAD' tem de ser dada na sala 3
domains[ESI3C[0]] = set((CR[3], CR[5])) # A UC 'IA' tem de ser dada na sala 3
domains[ESI3C[1]] = set((CR[3], CR[5])) # A UC 'PDM' tem de ser dada na sala 3
domains[ESI3C[2]] = set((CR[3], CR[5])) # A UC 'ISI' tem de ser dada na sala 3

domains[ESI3C[3]] = set((CR[4], CR[5])) # A UC 'SETR' tem de ser dada na sala 4
domains[ESI3C[4]] = set((CR[4], CR[5])) # A UC 'PA' tem de ser dada na sala 4

# Constraints
constraints = [
    #Rules
    Constraint((ESI1B), max_three),         # Máximo de 3 UCs por dia
    Constraint((ESI2B), max_three),         # Máximo de 3 UCs por dia
    Constraint((ESI3B), max_three),         # Máximo de 3 UCs por dia
    Constraint((ESI1C), min_one_online),    # Mínimo 1 UC online
    Constraint((ESI1C), max_two_online),    # Máximo 2 UCs online
    Constraint((ESI2C), min_one_online),    # Mínimo 1 UC online
    Constraint((ESI2C), max_two_online),    # Máximo 2 UCs online
    Constraint((ESI3C), min_one_online),    # Mínimo 1 UC online
    Constraint((ESI3C), max_two_online),    # Máximo 2 UCs online
    Constraint((ESI1B + ESI1C), space_between_ucs),  # Atribuir um bloco vazio entre uma UC online e presencial
    Constraint((ESI2B + ESI2C), space_between_ucs),  # Atribuir um bloco vazio entre uma UC online e presencial
    Constraint((ESI3B + ESI3C), space_between_ucs),  # Atribuir um bloco vazio entre uma UC online e presencial

    # Classes
    Constraint((ESI1B), all_diff_constraint), # Todas as UCs tê de ser em blocos diferentes
    Constraint((ESI2B), all_diff_constraint), # Todas as UCs tê de ser em blocos diferentes
    Constraint((ESI3B), all_diff_constraint), # Todas as UCs tê de ser em blocos diferentes

    # Teachers
    Constraint((ESI1B[3], ESI2B[0], ESI2B[1]), all_diff_constraint),    # O professor 'T2' tem de dar as UCs em blocos diferentes
    Constraint((ESI2B[4], ESI3B[0]), all_diff_constraint),              # O professor 'T4' tem de dar as UCs em blocos diferentes

    # ClassRooms
    Constraint((ESI1B[0], ESI1B[1], ESI2B[0]), all_diff_constraint),            # A sala 0 só pode ser atribuída uma vez em cada bloco
    Constraint((ESI1B[2], ESI1B[3], ESI2B[1]), all_diff_constraint),            # A sala 1 só pode ser atribuída uma vez em cada bloco
    Constraint((ESI1B[4], ESI2B[2], ESI2B[3]), all_diff_constraint),            # A sala 2 só pode ser atribuída uma vez em cada bloco
    Constraint((ESI2B[4], ESI3B[0], ESI3B[1], ESI3B[2]), all_diff_constraint),  # A sala 1 só pode ser atribuída uma vez em cada bloco
]

# Class scheduling
class_scheduling = NaryCSP(domains, constraints)

# print variables
print(class_scheduling.variables)

# Result
solver = ac_solver(class_scheduling, arc_heuristic=sat_up)
teachers = addTeachers(TESI1, solver)
teachers.update(addTeachers(TESI2, solver))
teachers.update(addTeachers(TESI3, solver))

# Output
finalOutput(solver, teachers)

#### Atribuição de dois blocos por UC para três turmas

No terceiro teste pretendíamos executar executar o algoritmo atribuindo a três turma dois blocos por UC, no entanto, quando atribuímos mais que dois blocos por UC em sete UCs às três turmas deixamos de obter solução.  
Mantivemos os sete professores, cada um responsável por diferentes UCs.  
Mantivemos, também, as cinco salas por onde poderão ser distribuídas as UCs, sendo que cada sala tem UCs especificas.

Tempo de conclusão do algoritmo: não encontrado.

In [ ]:
## Atribuição de dois blocos por UC para três turmas

schedules = set(range(1, 21))
domains = {}

## ClassRooms
# 0 -> PI, MD, POO | 1 -> C, RC, AMS | 2 -> LI, FF, PES | 3 -> AAD, IA, PDM, ISI | 4 -> SETR, PA
CR = [0, 1, 2, 3, 4, 50]

## Teachers
# T0 -> PI, LI | T1 -> MD, C | T2 -> RC, POO, AMS | T3 -> FF, PES | T4 -> AAD, IA | T5 -> PDM, ISI | T6 -> SETR, PA
TESI1 = ['T0.PI1', 'T0.LI1', 'T1.MD1', 'T1.C1', 'T2.RC1', 'T0.PI2', 'T1.MD2', 'T1.C2', 'T2.RC2', 'T0.LI2']       
TESI2 = ['T2.POO1', 'T2.AMS1', 'T3.FF1', 'T3.PES1', 'T4.AAD1', 'T2.POO2', 'T2.AMS2', 'T3.FF2', 'T3.PES2', 'T4.AAD2']  
TESI3 = ['T4.IA1', 'T5.PDM1', 'T5.ISI1', 'T6.SETR1' ,'T6.PA1', 'T4.IA2', 'T5.PDM2', 'T5.ISI2', 'T6.SETR2' ,'T6.PA2',]  


## VARIABLES
# Classes
    # 1º ano ESI
ESI1B = ['ESI1B.PI1', 'ESI1B.MD1', 'ESI1B.C1', 'ESI1B.RC1', 'ESI1B.LI1', 'ESI1B.PI2', 'ESI1B.MD2', 'ESI1B.C2', 'ESI1B.RC2', 'ESI1B.LI2']    # Armazenamento do bloco em que a UC é dada
ESI1C = ['ESI1C.PI1', 'ESI1C.MD1', 'ESI1C.C1', 'ESI1C.RC1', 'ESI1C.LI1', 'ESI1C.PI2', 'ESI1C.MD2', 'ESI1C.C2', 'ESI1C.RC2', 'ESI1C.LI2']    # Armazenamento da sala em que a UC é dada
    # 2º ano ESI
ESI2B = ['ESI2B.POO1', 'ESI2B.AMS1', 'ESI2B.FF1', 'ESI2B.PES1', 'ESI2B.AAD1', 'ESI2B.POO2', 'ESI2B.AMS2', 'ESI2B.FF2', 'ESI2B.PES2', 'ESI2B.AAD2']    # Armazenamento do bloco em que a UC é dada
ESI2C = ['ESI2C.POO1', 'ESI2C.AMS1', 'ESI2C.FF1', 'ESI2C.PES1', 'ESI2C.AAD1', 'ESI2C.POO2', 'ESI2C.AMS2', 'ESI2C.FF2', 'ESI2C.PES2', 'ESI2C.AAD2']    # Armazenamento da sala em que a UC é dada
    # 3º ano ESI
ESI3B = ['ESI3B.IA1', 'ESI3B.PDM1', 'ESI3B.ISI1', 'ESI3B.SETR1', 'ESI3B.PA1', 'ESI3B.IA2', 'ESI3B.PDM2', 'ESI3B.ISI2', 'ESI3B.SETR2', 'ESI3B.PA2']    # Armazenamento do bloco em que a UC é dada
ESI3C = ['ESI3C.IA1', 'ESI3C.PDM1', 'ESI3C.ISI1', 'ESI3C.SETR1', 'ESI3C.PA1', 'ESI3C.IA2', 'ESI3C.PDM2', 'ESI3C.ISI2', 'ESI3C.SETR2', 'ESI3C.PA2']    # Armazenamento da sala em que a UC é dada

## Atribuição de domínios às variáveis
variables = set (ESI1B + ESI2B + ESI3B)
for var in variables:
    domains[var] = schedules

# ClassRooms
domains[ESI1C[0]] = set((CR[0], CR[5])) # A UC 'PI1' tem de ser dada na sala 0
domains[ESI1C[1]] = set((CR[0], CR[5])) # A UC 'MD1' tem de ser dada na sala 0
domains[ESI1C[5]] = set((CR[0], CR[5])) # A UC 'PI2' tem de ser dada na sala 0
domains[ESI1C[6]] = set((CR[0], CR[5])) # A UC 'MD2' tem de ser dada na sala 0
domains[ESI2C[0]] = set((CR[0], CR[5])) # A UC 'POO1' tem de ser dada na sala 0
domains[ESI2C[5]] = set((CR[0], CR[5])) # A UC 'POO2' tem de ser dada na sala 0

domains[ESI1C[2]] = set((CR[1], CR[5])) # A UC 'C1' tem de ser dada na sala 1
domains[ESI1C[3]] = set((CR[1], CR[5])) # A UC 'RC1' tem de ser dada na sala 1
domains[ESI2C[1]] = set((CR[1], CR[5])) # A UC 'AMS1' tem de ser dada na sala 1
domains[ESI2C[6]] = set((CR[1], CR[5])) # A UC 'AMS2' tem de ser dada na sala 1
domains[ESI1C[7]] = set((CR[1], CR[5])) # A UC 'C2' tem de ser dada na sala 1
domains[ESI1C[8]] = set((CR[1], CR[5])) # A UC 'RC2' tem de ser dada na sala 1

domains[ESI1C[4]] = set((CR[2], CR[5])) # A UC 'LI1' tem de ser dada na sala 2
domains[ESI2C[2]] = set((CR[2], CR[5])) # A UC 'FF1' tem de ser dada na sala 2
domains[ESI2C[3]] = set((CR[2], CR[5])) # A UC 'PES1' tem de ser dada na sala 2
domains[ESI1C[9]] = set((CR[2], CR[5])) # A UC 'LI2' tem de ser dada na sala 2
domains[ESI2C[7]] = set((CR[2], CR[5])) # A UC 'FF2' tem de ser dada na sala 2
domains[ESI2C[8]] = set((CR[2], CR[5])) # A UC 'PES2' tem de ser dada na sala 2

domains[ESI2C[4]] = set((CR[3], CR[5])) # A UC 'AAD1' tem de ser dada na sala 3
domains[ESI3C[0]] = set((CR[3], CR[5])) # A UC 'IA1' tem de ser dada na sala 3
domains[ESI3C[1]] = set((CR[3], CR[5])) # A UC 'PDM1' tem de ser dada na sala 3
domains[ESI3C[2]] = set((CR[3], CR[5])) # A UC 'ISI1' tem de ser dada na sala 3
domains[ESI3C[5]] = set((CR[3], CR[5])) # A UC 'IA2' tem de ser dada na sala 3
domains[ESI3C[6]] = set((CR[3], CR[5])) # A UC 'PDM2' tem de ser dada na sala 3
domains[ESI2C[9]] = set((CR[3], CR[5])) # A UC 'AAD2' tem de ser dada na sala 3
domains[ESI3C[7]] = set((CR[3], CR[5])) # A UC 'ISI1' tem de ser dada na sala 3

domains[ESI3C[3]] = set((CR[4], CR[5])) # A UC 'SETR1' tem de ser dada na sala 4
domains[ESI3C[4]] = set((CR[4], CR[5])) # A UC 'PA1' tem de ser dada na sala 4
domains[ESI3C[8]] = set((CR[4], CR[5])) # A UC 'SETR2' tem de ser dada na sala 4
domains[ESI3C[9]] = set((CR[4], CR[5])) # A UC 'PA1' tem de ser dada na sala 4

# Constraints
constraints = [
    #Rules
    Constraint((ESI1B), max_three),         # Máximo de 3 UCs por dia
    Constraint((ESI2B), max_three),         # Máximo de 3 UCs por dia
    Constraint((ESI3B), max_three),         # Máximo de 3 UCs por dia
    Constraint((ESI1C), min_one_online),    # Mínimo 1 UC online
    Constraint((ESI1C), max_two_online),    # Máximo 2 UCs online
    Constraint((ESI2C), min_one_online),    # Mínimo 1 UC online
    Constraint((ESI2C), max_two_online),    # Máximo 2 UCs online
    Constraint((ESI3C), min_one_online),    # Mínimo 1 UC online
    Constraint((ESI3C), max_two_online),    # Máximo 2 UCs online
    Constraint((ESI1B + ESI1C), space_between_ucs),  # Atribuir um bloco vazio entre uma UC online e presencial
    Constraint((ESI2B + ESI2C), space_between_ucs),  # Atribuir um bloco vazio entre uma UC online e presencial
    Constraint((ESI3B + ESI3C), space_between_ucs),  # Atribuir um bloco vazio entre uma UC online e presencial

    # Classes
    Constraint((ESI1B), all_diff_constraint), # Todas as UCs tê de ser em blocos diferentes
    Constraint((ESI2B), all_diff_constraint), # Todas as UCs tê de ser em blocos diferentes
    Constraint((ESI3B), all_diff_constraint), # Todas as UCs tê de ser em blocos diferentes

    # Teachers
    Constraint((ESI1B[0], ESI1B[4], ESI1B[5], ESI1B[9]), all_diff_constraint),                      # O professor 'T0' tem de dar as UCs em blocos diferentes          
    Constraint((ESI1B[1], ESI1B[2], ESI1B[6], ESI1B[7]), all_diff_constraint),                      # O professor 'T1' tem de dar as UCs em blocos diferentes                                      
    Constraint((ESI1B[3], ESI2B[0], ESI2B[1], ESI2B[5], ESI2B[6], ESI1B[8]), all_diff_constraint),  # O professor 'T2' tem de dar as UCs em blocos diferentes
    Constraint((ESI2B[2], ESI2B[3], ESI2B[7], ESI2B[8]), all_diff_constraint),                      # O professor 'T3' tem de dar as UCs em blocos diferentes
    Constraint((ESI2B[4], ESI3B[0], ESI3B[5], ESI2B[9]), all_diff_constraint),                      # O professor 'T4' tem de dar as UCs em blocos diferentes
    Constraint((ESI3B[1], ESI3B[2], ESI3B[6], ESI3B[7]), all_diff_constraint),                      # O professor 'T5' tem de dar as UCs em blocos diferentes
    Constraint((ESI3B[3], ESI3B[4], ESI3B[8], ESI3B[9]), all_diff_constraint),                      # O professor 'T6' tem de dar as UCs em blocos diferentes

    # ClassRooms
    Constraint((ESI1B[0], ESI1B[1], ESI1B[5], ESI1B[6], ESI2B[0], ESI2B[5]), all_diff_constraint),              # A sala 0 só pode ser atribuída uma vez em cada bloco
    Constraint((ESI1B[2], ESI1B[3], ESI2B[1], ESI2B[6], ESI1B[7], ESI1B[8]), all_diff_constraint),              # A sala 1 só pode ser atribuída uma vez em cada bloco
    Constraint((ESI1B[4], ESI2B[2], ESI2B[3], ESI1B[9], ESI2B[7], ESI2B[8], ESI2B[9]), all_diff_constraint),    # A sala 2 só pode ser atribuída uma vez em cada bloco
    Constraint((ESI2B[4], ESI3B[0], ESI3B[1], ESI3B[2], ESI3B[5], ESI3B[6], ESI3B[7]), all_diff_constraint),    # A sala 3 só pode ser atribuída uma vez em cada bloco
    Constraint((ESI3B[3], ESI3B[4], ESI3B[8], ESI3B[9]), all_diff_constraint),                                  # A sala 4 só pode ser atribuída uma vez em cada bloco
]

# Class scheduling
class_scheduling = NaryCSP(domains, constraints)

# print variables
print(class_scheduling.variables)

# Result
solver = ac_solver(class_scheduling, arc_heuristic=sat_up)
teachers = addTeachers(TESI1, solver)
teachers.update(addTeachers(TESI2, solver))
teachers.update(addTeachers(TESI3, solver))

# Output
finalOutput(solver, teachers)